<a href="https://colab.research.google.com/github/ImineAmazigh/MadLibs/blob/main/PricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
#these are just libraries, that will be used, if you want to install more go to terminal or tell me
import tensorflow as tf
import optuna
import pandas as pd
import numpy as np
import re

from google.colab import files

In [86]:
#our code starts here
df = pd.read_csv('/content/train_car.csv')
print(df)
id_train = df['id'].to_numpy(dtype=np.int32)
print(id_train)
model_train = df['model'].to_numpy(dtype=str)
print(model_train)
mileage_km_train = df['mileage_km'].to_numpy(dtype = np.int32)
print(mileage_km_train)
year_train = df['year'].to_numpy(dtype=np.float16)
print(year_train)
transmission_train = df['transmission'].to_numpy(dtype=str)
print(transmission_train)
fuel_type_train = df['fuel_type'].to_numpy(dtype=str)
print(fuel_type_train)
listing_date_train = df['listing_date']
to_date_var = pd.to_datetime(listing_date_train)
df['listing_date'] = to_date_var.dt.year + (to_date_var.dt.dayofyear - 1)/365
listing_date_train = df['listing_date'].to_numpy(dtype=np.float64)
print(listing_date_train)

# Define a function to convert string to ASCII sum
def string_to_ascii_sum(s):
    return sum(ord(char) for char in s) if isinstance(s, str) else s

# Convert string arrays to numeric using ASCII sum
model_train_numeric = np.array([string_to_ascii_sum(s) for s in model_train], dtype=np.float32)
transmission_train_numeric = np.array([string_to_ascii_sum(s) for s in transmission_train], dtype=np.float32)
fuel_type_train_numeric = np.array([string_to_ascii_sum(s) for s in fuel_type_train], dtype=np.float32)

#use it later .iloc[:7666]
priceMatrixHolder = np.array([df['price']])

# Recreate matrixHolder with numeric values
matrixHolder = np.array([
    id_train,
    model_train_numeric,
    mileage_km_train,
    year_train,
    transmission_train_numeric,
    fuel_type_train_numeric,
    listing_date_train
])
print(matrixHolder)
print("we'll separate our data to 80% training and 20% testing")

         id                     model  mileage_km  year transmission  \
0      5895                CITROEN C3     45000.0  2021     Manuelle   
1      2926                    BMW X3    149000.0  2019  Automatique   
2      7027                   AUDI Q5    169000.0  2015  Automatique   
3      1765              DACIA Duster    235000.0  2021     Manuelle   
4      8931                OPEL Corsa     70000.0  2021     Manuelle   
...     ...                       ...         ...   ...          ...   
9577   5283  MERCEDES-BENZ Classe cla    120000.0  2017  Automatique   
9578  13643             DACIA Sandero       115.0  2018     Manuelle   
9579   5483                   AUDI Q3    180000.0  2012  Automatique   
9580    879                   AUDI Q3    109000.0  2022  Automatique   
9581   7390      MITSUBISHI Outlander    159000.0  2015  Automatique   

     fuel_type listing_date  tax_hp  doors   price  
0       Diesel   2025-02-14     6.0    5.0  142000  
1       Diesel   2025-10-29  

/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(values, dtype=dtype)


In [87]:
print(year_train.dtype)

float16


In [ ]:
def objective(trial) :
  lrBest = trial.suggest_float('trial', 1e-4, 1e-1, log=True)
  echosBest = trial.suggest_int('echosBest', 1, 64)
  n1Best = trial.suggest_int('n1Best',1 ,64)
  n2Best = trial.suggest_int('n2Best',1,32)
  model = tf.keras.Sequential([tf.keras.Input(shape= (7,)), # Changed shape to match matrixHolder.T
                               tf.keras.layers.Dense(n1Best, activation='relu'),
                               tf.keras.layers.Dense(n2Best,activation='relu'),
                               tf.keras.layers.Dense(1)])
  model.compile(optimizer='adamw',
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
                loss='mse')

  X = matrixHolder.T
  y = priceMatrixHolder.T

  X_numeric = np.asarray(X, dtype=np.float32)
  y_numeric = np.asarray(y, dtype=np.float32)

  model.fit(X_numeric, y_numeric, epochs=echosBest, verbose=0)
  return model.history.history['root_mean_squared_error'][-1]
new_study = optuna.create_study()
new_study.optimize(objective, n_trials=2)

[I 2026-02-13 00:18:04,116] A new study created in memory with name: no-name-15858af8-8341-4396-9183-6ed145342c5a


In [ ]:
from google.colab import drive
drive.mount('/content/drive')